# BHAI VAE Paper Figures

Generate all figures for the paper.

**Figures:**
1. `fig_lily_expedition_map.png` - IODP expedition locations (cartopy)
2. `fig_lily_dataset.png` - Dataset overview (3 panels)
3. `fig_lily_lithology_counts.png` - Lithology distribution
4. `fig_lily_variables_dist.png` - Input variable distributions with KDE
5. `fig_reconstruction_scatter.png` - Reconstruction quality (2 rows × 6 cols)
6. `fig_roc_comparison.png` - ROC curves (2 panels: SVM vs Classification head)
7. `fig_umap_lithology.png` - UMAP (2 panels: unsup vs semi-sup)
8. `fig_r2_unsup_vs_semi.png` - R² comparison with bootstrapped CIs

**NOT generated:** `neural-network-diagram-5.pdf`

**Style:**
- No top/right spines
- UMAP: no spines at all
- No titles (labels in boxes where needed)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from pathlib import Path
from scipy import stats
from collections import Counter
import sys

sys.path.insert(0, '..')
from models.vae import VAE, SemiSupervisedVAE, DistributionAwareScaler

%matplotlib inline

# Global settings
plt.rcParams['figure.dpi'] = 150
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False

FEATURE_COLS = ['Bulk density (GRA)', 'Magnetic susceptibility (instr. units)', 
                'NGR total counts (cps)', 'R', 'G', 'B']
FEATURE_LABELS = ['Bulk Density', 'Mag. Susc.', 'NGR', 'R', 'G', 'B']

In [ ]:
# Paths
DATA_DIR = Path('../data')
MODEL_DIR = Path('../models')
OUTPUT_DIR = Path('../figures')
OUTPUT_DIR.mkdir(exist_ok=True)

In [ ]:
# Load data
print("Loading data...")
train_df = pd.read_csv(DATA_DIR / 'vae_training_data_v2_20cm.csv')
train_df['Expedition'] = train_df['Borehole_ID'].str.split('-').str[0]

X_raw = train_df[FEATURE_COLS].values
valid_mask = ~np.isnan(X_raw).any(axis=1)
train_df_valid = train_df[valid_mask].reset_index(drop=True)

scaler = DistributionAwareScaler()
X_scaled = scaler.fit_transform(X_raw[valid_mask])

print(f"Samples: {len(train_df):,}")
print(f"Valid: {len(train_df_valid):,}")
print(f"Expeditions: {train_df['Expedition'].nunique()}")
print(f"Lithologies: {train_df['Principal'].nunique()}")

In [ ]:
# Load models
print("Loading models...")
model_unsup = VAE(input_dim=6, latent_dim=10)
model_unsup.load_state_dict(torch.load(MODEL_DIR / 'unsup.pt', map_location='cpu'))
model_unsup.eval()

model_semisup = SemiSupervisedVAE(input_dim=6, latent_dim=10, n_classes=139)
model_semisup.load_state_dict(torch.load(MODEL_DIR / 'semisup.pt', map_location='cpu'))
model_semisup.eval()

print("Generating embeddings...")
with torch.no_grad():
    X_t = torch.FloatTensor(X_scaled)
    emb_unsup = model_unsup.get_embeddings(X_t).numpy()
    emb_semisup = model_semisup.get_embeddings(X_t).numpy()
    recon_unsup, _, _ = model_unsup(X_t)
    recon_semisup, _, _, _ = model_semisup(X_t)
    recon_unsup = recon_unsup.numpy()
    recon_semisup = recon_semisup.numpy()

print(f"Embeddings: {emb_unsup.shape}")

---
## Figure 1: Expedition Map

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Expedition coordinates with label offsets (lat, lon, lat_off, lon_off)
EXPEDITION_COORDS = {
    '346': (40.0, 138.0, 5, 0), '349': (18.0, 117.0, -5, -8),
    '350': (32.0, 140.0, 5, 5), '351': (30.0, 140.0, -5, 8),
    '352': (28.5, 141.0, 0, 8), '353': (19.0, 88.0, 5, 0),
    '354': (8.0, 88.0, -5, 0), '355': (17.0, 68.0, 5, -5),
    '356': (-18.0, 115.0, 5, 0), '359': (5.0, 73.0, -5, 0),
    '360': (-32.0, 57.0, 0, -8), '361': (-35.0, 25.0, 5, 0),
    '362': (3.0, 96.0, -5, 0), '363': (-2.0, 142.0, 5, 0),
    '366': (18.0, 147.0, 0, 8), '367': (19.0, 116.0, 5, 5),
    '368': (19.5, 116.5, -8, 0), '368X': (20.0, 117.0, 8, 0),
    '369': (-34.0, 113.0, -5, 0), '371': (-35.0, 165.0, 5, -5),
    '372': (-38.0, 168.0, -5, 5), '374': (-75.0, 175.0, 5, 0),
    '375': (-39.0, 178.0, 0, 8), '376': (-37.0, 177.0, 5, 5),
    '379': (-57.0, -94.0, 5, 0),
}

exp_counts = train_df['Expedition'].value_counts().to_dict()

fig = plt.figure(figsize=(14, 8))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.Robinson(central_longitude=104))

ax.set_global()
ax.add_feature(cfeature.LAND, facecolor='#f5f5dc', edgecolor='none')
ax.add_feature(cfeature.OCEAN, facecolor='#e6f3ff')
ax.add_feature(cfeature.COASTLINE, linewidth=0.5, edgecolor='gray')
ax.add_feature(cfeature.BORDERS, linewidth=0.3, edgecolor='gray', linestyle=':')
ax.gridlines(draw_labels=False, linewidth=0.3, color='gray', alpha=0.5)

max_count = max(exp_counts.values())
for exp, coords in EXPEDITION_COORDS.items():
    lat, lon, lat_off, lon_off = coords
    if exp in exp_counts:
        count = exp_counts[exp]
        size = 50 + 350 * np.sqrt(count / max_count)
        ax.scatter(lon, lat, s=size, c='#c44e52', alpha=0.8,
                  edgecolors='darkred', linewidth=0.5,
                  transform=ccrs.PlateCarree(), zorder=5)
        ax.text(lon + lon_off, lat + lat_off, exp, fontsize=7, fontweight='bold',
               transform=ccrs.PlateCarree(), zorder=6)

# Title in box
ax.text(0.5, 0.92, 'IODP Boreholes from the LILY Dataset', 
        transform=ax.transAxes, fontsize=14, fontweight='bold',
        ha='center', va='center',
        bbox=dict(boxstyle='round,pad=0.5', facecolor='white', edgecolor='gray', alpha=0.9))

# Legend with box
legend_ax = fig.add_axes([0.80, 0.55, 0.16, 0.28])
legend_ax.patch.set_facecolor('white')
legend_ax.patch.set_edgecolor('gray')
legend_ax.patch.set_linewidth(1.5)
legend_ax.set_frame_on(True)
legend_ax.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
legend_ax.text(0.5, 0.88, 'Samples', ha='center', fontsize=11, fontweight='bold')
for ss, yp in [(1000, 0.65), (10000, 0.4), (40000, 0.15)]:
    size = 50 + 350 * np.sqrt(ss / max_count)
    legend_ax.scatter(0.3, yp, s=size, c='#c44e52', alpha=0.8, edgecolors='darkred', linewidth=0.5)
    legend_ax.text(0.55, yp, f'{ss:,}', va='center', fontsize=10)
legend_ax.set_xlim(0, 1)
legend_ax.set_ylim(0, 1)

plt.tight_layout()
fig.savefig(OUTPUT_DIR / 'fig_lily_expedition_map.png', dpi=300, bbox_inches='tight', facecolor='white')
print("Saved fig_lily_expedition_map.png")
plt.show()

---
## Figure 2: Dataset Overview

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Panel 1: Samples per expedition - LIME GREEN
ax = axes[0]
exp_counts_sorted = train_df['Expedition'].value_counts().sort_values()
ax.barh(exp_counts_sorted.index, exp_counts_sorted.values, color='limegreen')
ax.set_xlabel('Number of Samples', fontsize=11)
ax.set_ylabel('Expedition', fontsize=11)

# Panel 2: Depth distribution - DODGER BLUE
ax = axes[1]
ax.hist(train_df['Depth_Bin'].values, bins=50, color='dodgerblue', edgecolor='white', alpha=0.9)
ax.set_xlabel('Depth (m)', fontsize=11)
ax.set_ylabel('Count', fontsize=11)

# Panel 3: Boreholes per expedition - ORANGE
ax = axes[2]
borehole_counts = train_df.groupby('Expedition')['Borehole_ID'].nunique().sort_values()
ax.barh(borehole_counts.index, borehole_counts.values, color='#ff7f0e')
ax.set_xlabel('Number of Boreholes', fontsize=11)
ax.set_ylabel('Expedition', fontsize=11)

plt.tight_layout()
fig.savefig(OUTPUT_DIR / 'fig_lily_dataset.png', dpi=300, bbox_inches='tight')
print("Saved fig_lily_dataset.png")
plt.show()

---
## Figure 3: Lithology Counts

Use existing figure from bhai-analysis (steel blue, all lithologies, count labels)

In [ ]:
import shutil
src = Path('/home/mnky9800n/clawd/bhai-analysis/fig_lily_lithology_counts.png')
if src.exists():
    shutil.copy(src, OUTPUT_DIR / 'fig_lily_lithology_counts.png')
    print("Copied fig_lily_lithology_counts.png from bhai-analysis")
else:
    print("Source not found - generate manually")

---
## Figure 4: Variable Distributions (with KDE)

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(14, 8))
axes = axes.flatten()
colors = ['#2c3e50', '#8e44ad', '#16a085', '#e74c3c', '#27ae60', '#3498db']

for i, (col, label, color) in enumerate(zip(FEATURE_COLS, FEATURE_LABELS, colors)):
    ax = axes[i]
    data = train_df[col].dropna().values
    
    ax.hist(data, bins=50, color=color, edgecolor='white', alpha=0.7, density=True)
    
    # KDE curve
    kde = stats.gaussian_kde(data)
    x_range = np.linspace(data.min(), data.max(), 200)
    ax.plot(x_range, kde(x_range), color='black', linewidth=2)
    
    ax.set_xlabel(label, fontsize=11)
    ax.set_ylabel('Density', fontsize=11)

plt.tight_layout()
fig.savefig(OUTPUT_DIR / 'fig_lily_variables_dist.png', dpi=300, bbox_inches='tight')
print("Saved fig_lily_variables_dist.png")
plt.show()

---
## Figure 5: Reconstruction Scatter

In [ ]:
def r2_score(y_true, y_pred):
    ss_res = np.sum((y_true - y_pred)**2)
    ss_tot = np.sum((y_true - np.mean(y_true))**2)
    return 1 - ss_res / ss_tot

fig, axes = plt.subplots(2, 6, figsize=(18, 8))
colors = ['#2c3e50', '#8e44ad', '#16a085', '#e74c3c', '#27ae60', '#3498db']

np.random.seed(42)
n_plot = min(10000, len(X_scaled))
plot_idx = np.random.choice(len(X_scaled), n_plot, replace=False)

for col_idx, (label, color) in enumerate(zip(FEATURE_LABELS, colors)):
    for row_idx, (pred, model_name) in enumerate([(recon_unsup, 'Unsupervised'), 
                                                   (recon_semisup, 'Semi-supervised')]):
        ax = axes[row_idx, col_idx]
        true_vals = X_scaled[plot_idx, col_idx]
        pred_vals = pred[plot_idx, col_idx]
        r2 = r2_score(X_scaled[:, col_idx], pred[:, col_idx])
        
        ax.scatter(true_vals, pred_vals, alpha=0.3, s=1, c=color)
        lims = [min(true_vals.min(), pred_vals.min()) - 0.5, 
                max(true_vals.max(), pred_vals.max()) + 0.5]
        ax.plot(lims, lims, '--', color='gray', alpha=0.5, linewidth=1.5)
        ax.set_xlim(lims)
        ax.set_ylim(lims)
        
        # Title + R² in box
        ax.text(0.05, 0.95, f'{label}\nR²={r2:.3f}', transform=ax.transAxes, 
                fontsize=10, va='top', fontweight='bold',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.9))
        
        if col_idx == 0:
            ax.set_ylabel(f'{model_name}\nPredicted', fontsize=10)
        if row_idx == 1:
            ax.set_xlabel('True', fontsize=10)

plt.tight_layout()
fig.savefig(OUTPUT_DIR / 'fig_reconstruction_scatter.png', dpi=300, bbox_inches='tight')
print("Saved fig_reconstruction_scatter.png")
plt.show()

---
## Figure 6: ROC Comparison (SVM vs Classification Head)

In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

# Get top 10 lithologies
top_liths = train_df_valid['Principal'].value_counts().head(10).index.tolist()
mask = train_df_valid['Principal'].isin(top_liths)

X_scaled_sub = X_scaled[mask]
labels = train_df_valid.loc[mask, 'Principal'].values
label_to_idx = {l: i for i, l in enumerate(top_liths)}
y = np.array([label_to_idx[l] for l in labels])

# Subsample
np.random.seed(42)
n_max = 20000
if len(y) > n_max:
    idx = np.random.choice(len(y), n_max, replace=False)
    X_scaled_sub = X_scaled_sub[idx]
    labels = labels[idx]
    y = y[idx]

X_train, X_test, y_train, y_test = train_test_split(X_scaled_sub, y, test_size=0.3, random_state=42, stratify=y)

# Get embeddings
with torch.no_grad():
    X_train_t = torch.FloatTensor(X_train)
    X_test_t = torch.FloatTensor(X_test)
    emb_train_u = model_unsup.get_embeddings(X_train_t).numpy()
    emb_test_u = model_unsup.get_embeddings(X_test_t).numpy()
    emb_test_s = model_semisup.get_embeddings(X_test_t)
    
    # Classification head predictions
    logits = model_semisup.classifier(emb_test_s)
    prob_semisup_full = F.softmax(logits, dim=1).numpy()

print("Training SVM...")
svm = SVC(kernel='rbf', probability=True, random_state=42)
svm.fit(emb_train_u, y_train)
prob_unsup = svm.predict_proba(emb_test_u)

# Get semi-sup probabilities for top 10 classes
all_labels = train_df_valid['Principal'].values
unique_all = sorted(np.unique(all_labels))
full_label_to_idx = {l: i for i, l in enumerate(unique_all)}
top_lith_indices = [full_label_to_idx[l] for l in top_liths]
prob_semisup = prob_semisup_full[:, top_lith_indices]
prob_semisup = prob_semisup / prob_semisup.sum(axis=1, keepdims=True)

y_test_bin = label_binarize(y_test, classes=list(range(10)))
colors = plt.cm.tab10(np.linspace(0, 1, 10))

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Left: Unsupervised (SVM)
ax = axes[0]
aucs_u = []
for i, lith in enumerate(top_liths):
    fpr, tpr, _ = roc_curve(y_test_bin[:, i], prob_unsup[:, i])
    auc_val = auc(fpr, tpr)
    aucs_u.append(auc_val)
    ax.plot(fpr, tpr, color=colors[i], lw=1.5, label=f'{lith} ({auc_val:.2f})')
ax.plot([0, 1], [0, 1], 'k--', lw=1, alpha=0.5)
ax.set_xlabel('False Positive Rate', fontsize=11)
ax.set_ylabel('True Positive Rate', fontsize=11)
ax.set_title(f'v2.6.7 Unsupervised\n(SVM on embeddings, Mean AUC={np.mean(aucs_u):.2f})', fontsize=11)
ax.legend(loc='lower right', fontsize=8)
ax.set_xlim([0, 1]); ax.set_ylim([0, 1.02])

# Right: Semi-supervised (Classification head)
ax = axes[1]
aucs_s = []
for i, lith in enumerate(top_liths):
    fpr, tpr, _ = roc_curve(y_test_bin[:, i], prob_semisup[:, i])
    auc_val = auc(fpr, tpr)
    aucs_s.append(auc_val)
    ax.plot(fpr, tpr, color=colors[i], lw=1.5, label=f'{lith} ({auc_val:.2f})')
ax.plot([0, 1], [0, 1], 'k--', lw=1, alpha=0.5)
ax.set_xlabel('False Positive Rate', fontsize=11)
ax.set_ylabel('True Positive Rate', fontsize=11)
ax.set_title(f'v2.14 Semi-supervised\n(Classification head, Mean AUC={np.mean(aucs_s):.2f})', fontsize=11)
ax.legend(loc='lower right', fontsize=8)
ax.set_xlim([0, 1]); ax.set_ylim([0, 1.02])

plt.tight_layout()
fig.savefig(OUTPUT_DIR / 'fig_roc_comparison.png', dpi=300, bbox_inches='tight')
print("Saved fig_roc_comparison.png")
plt.show()

---
## Figure 7: UMAP (Two Panels)

In [ ]:
import umap

# Subsample
np.random.seed(42)
max_samples = 30000
idx = np.random.choice(len(emb_unsup), max_samples, replace=False)
emb_unsup_sub = emb_unsup[idx]
emb_semisup_sub = emb_semisup[idx]
labels_sub = train_df_valid['Principal'].values[idx]

print("Computing UMAP for unsupervised...")
reducer_u = umap.UMAP(n_neighbors=15, min_dist=0.1, random_state=42, n_jobs=1)
proj_unsup = reducer_u.fit_transform(emb_unsup_sub)

print("Computing UMAP for semi-supervised...")
reducer_s = umap.UMAP(n_neighbors=15, min_dist=0.1, random_state=42, n_jobs=1)
proj_semisup = reducer_s.fit_transform(emb_semisup_sub)

# Top 15 lithologies
lith_counts = Counter(labels_sub)
top_liths_umap = [l for l, _ in lith_counts.most_common(15)]

fig, axes = plt.subplots(1, 2, figsize=(16, 8))
cmap = plt.cm.tab20
colors_map = {lith: cmap(i / 15) for i, lith in enumerate(top_liths_umap)}

for ax, proj, title in [(axes[0], proj_unsup, 'Unsupervised'), 
                         (axes[1], proj_semisup, 'Semi-supervised')]:
    other_mask = ~np.isin(labels_sub, top_liths_umap)
    ax.scatter(proj[other_mask, 0], proj[other_mask, 1], c='#cccccc', s=1, alpha=0.3, rasterized=True)
    
    for lith in top_liths_umap:
        m = labels_sub == lith
        ax.scatter(proj[m, 0], proj[m, 1], c=[colors_map[lith]], s=5, alpha=0.6, rasterized=True)
    
    ax.set_title(title, fontsize=14, fontweight='bold')
    # NO SPINES
    for spine in ax.spines.values(): spine.set_visible(False)
    ax.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)

# Shared legend at bottom
legend_elements = [Line2D([0], [0], marker='o', color='w', markerfacecolor='#cccccc', markersize=8, label='Other')]
for lith in top_liths_umap:
    legend_elements.append(Line2D([0], [0], marker='o', color='w', markerfacecolor=colors_map[lith], markersize=8, label=lith))
fig.legend(handles=legend_elements, loc='center', bbox_to_anchor=(0.5, -0.02), ncol=8, fontsize=9, frameon=False)

plt.tight_layout()
plt.subplots_adjust(bottom=0.12)
fig.savefig(OUTPUT_DIR / 'fig_umap_lithology.png', dpi=300, bbox_inches='tight')
print("Saved fig_umap_lithology.png")
plt.show()

---
## Figure 8: R² Comparison with Bootstrapped CIs

Load pre-computed bootstrap results (from `run_bootstrap_1337.py`)

In [ ]:
# Load bootstrap results
bootstrap_file = DATA_DIR / 'zeroshot_bootstrap_1337.csv'
if not bootstrap_file.exists():
    bootstrap_file = DATA_DIR / 'zeroshot_results_full.csv'  # Fallback
    print("Using fallback (no bootstrap CIs)")

df = pd.read_csv(bootstrap_file)
print(f"Loaded {len(df)} variables")

# Filter
df = df[(df['r2_v267'] > -1) & (df['r2_v214'] > -1) & 
        (df['r2_v267'] < 1.1) & (df['r2_v214'] < 1.1)]
print(f"After filtering: {len(df)} variables")

fig, ax = plt.subplots(figsize=(10, 10))

x = df['r2_v214'].values
y = df['r2_v267'].values
n_samples = df['n_samples'].values

# Check if CIs exist
has_ci = 'r2_v214_lo' in df.columns

sizes = 20 + 80 * (np.log10(np.clip(n_samples, 100, 1e6)) - 2) / 4
c_semi, c_unsup = '#20B2AA', '#F08080'
colors = [c_semi if xi > yi else c_unsup for xi, yi in zip(x, y)]

for i in range(len(x)):
    if has_ci:
        xerr = [[max(0, x[i] - df['r2_v214_lo'].iloc[i])], [max(0, df['r2_v214_hi'].iloc[i] - x[i])]]
        yerr = [[max(0, y[i] - df['r2_v267_lo'].iloc[i])], [max(0, df['r2_v267_hi'].iloc[i] - y[i])]]
        ax.errorbar(x[i], y[i], xerr=xerr, yerr=yerr, fmt='o', 
                    markersize=np.sqrt(sizes[i]/3), color=colors[i], alpha=0.6,
                    ecolor=colors[i], elinewidth=0.5, capsize=0)
    else:
        ax.scatter(x[i], y[i], s=sizes[i], c=colors[i], alpha=0.6, edgecolors='white', linewidth=0.5)

ax.plot([-0.2, 1.05], [-0.2, 1.05], 'k--', lw=1.5, alpha=0.5)
ax.set_xlabel('Semi-supervised R²', fontsize=14)
ax.set_ylabel('Unsupervised R²', fontsize=14)
ax.set_xlim(-0.2, 1.05)
ax.set_ylim(-0.2, 1.05)
ax.set_aspect('equal')

legend_elements = [
    Line2D([0], [0], marker='o', color='w', markerfacecolor=c_semi, markersize=10, label='Semi-supervised higher'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor=c_unsup, markersize=10, label='Unsupervised higher'),
]
ax.legend(handles=legend_elements, loc='lower right', fontsize=11)

# Small size legend
sax = fig.add_axes([0.15, 0.78, 0.1, 0.12])
sax.axis('off')
for ss, lab, yp in [(100, '100', 0.7), (10000, '10k', 0.2)]:
    sax.scatter(0.3, yp, s=20 + 80 * (np.log10(ss) - 2) / 4, c='gray', alpha=0.7)
    sax.text(0.6, yp, lab, va='center', fontsize=9)
sax.text(0.45, 0.95, 'n', ha='center', fontsize=10, fontweight='bold')
sax.set_xlim(0, 1); sax.set_ylim(0, 1)

semi_wins = sum(1 for xi, yi in zip(x, y) if xi > yi)
print(f"Semi-supervised wins: {semi_wins}/{len(x)}")

plt.tight_layout()
fig.savefig(OUTPUT_DIR / 'fig_r2_unsup_vs_semi.png', dpi=300, bbox_inches='tight')
print("Saved fig_r2_unsup_vs_semi.png")
plt.show()

---
## Summary

In [ ]:
import os

print("\n" + "="*60)
print("GENERATED FIGURES")
print("="*60)

expected = [
    'fig_lily_expedition_map.png',
    'fig_lily_dataset.png',
    'fig_lily_lithology_counts.png',
    'fig_lily_variables_dist.png',
    'fig_reconstruction_scatter.png',
    'fig_roc_comparison.png',
    'fig_umap_lithology.png',
    'fig_r2_unsup_vs_semi.png',
]

for name in expected:
    path = OUTPUT_DIR / name
    if path.exists():
        size = os.path.getsize(path) / 1024
        print(f"  ✓ {name}: {size:.1f} KB")
    else:
        print(f"  ✗ {name}: MISSING")

print("\nNOT generated (external):")
print("  - neural-network-diagram-5.pdf")
print("="*60)